# GoogLeNet V4 

详细论文参考：[Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning](https://arxiv.org/abs/1602.07261)

## Inception-V4 

#### Inception-V4网络的架构：

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-18.png" width="400">

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet import autograd

ctx = mx.cpu()

import matplotlib as mlt
import matplotlib.pyplot as plt

In [2]:
def BN_ReLU_Conv(channels, kernel_size, strides=1, padding=0):
    net = gluon.nn.Sequential()
    with net.name_scope():
        net.add(
            gluon.nn.BatchNorm(axis=1),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels, kernel_size=kernel_size, strides=strides, padding=padding)
        ) 
    return net

In [3]:
def Stem(X):
    # path1
    b1 = gluon.nn.Sequential()
    with b1.name_scope():
        b1.add(  
            BN_ReLU_Conv(32, kernel_size=3, strides=2),
            BN_ReLU_Conv(32, kernel_size=3),
            BN_ReLU_Conv(64, kernel_size=3, padding=1),        
        ) 
    conv1 = BN_ReLU_Conv(96, kernel_size=3, strides=2)
    maxp1 = gluon.nn.MaxPool2D(pool_size=2)
    path1 = nd.concat(conv1(b1(X)), maxp1(b1(X)), dim=1)
    
    # path2 
    b2 = gluon.nn.Sequential()
    with b2.name_scope():
        b2.add(
            BN_ReLU_Conv(64, kernel_size=1),
            BN_ReLU_Conv(96, kernel_size=3)
        )
    b3 = gluon.nn.Sequential()
    with b3.name_scope():
        b3.add(
            BN_ReLU_Conv(64, kernel_size=1),
            BN_ReLU_Conv(64, kernel_size=(7,1), padding=(3,0)),
            BN_ReLU_Conv(64, kernel_size=(1,7), padding=(0,3)),
            BN_ReLU_Conv(96, kernel_size=3)
        )
    path2 = nd.concat(b2(path1), b3(path1), dim=1)
    
    # path3
    conv2 = BN_ReLU_Conv(192, kernel_size=3, strides=2)
    maxp2 = gluon.nn.MaxPool2D(pool_size=2)
    path3 = nd.concat(conv2(path2), maxp2(path2), dim=1)
    
    return path3

In [4]:
class Stem(gluon.Block):
    def __init__(self, n1, n2, n3, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        # path1
        self.b1 = gluon.nn.Sequential()
        self.b1.add(  
            BN_ReLU_Conv(n1[0], kernel_size=3, strides=2),
            BN_ReLU_Conv(n1[1], kernel_size=3),
            BN_ReLU_Conv(n1[2], kernel_size=3, padding=1),   
        )
        self.conv1 = BN_ReLU_Conv(n1[3], kernel_size=3, strides=2)
        self.maxp1 = gluon.nn.MaxPool2D(pool_size=2)
        
        # path2
        self.b2 = gluon.nn.Sequential()
        self.b2.add(
            BN_ReLU_Conv(n2[0][0], kernel_size=1),
            BN_ReLU_Conv(n2[0][1], kernel_size=3)
        )
        self.b3 = gluon.nn.Sequential()
        self.b3.add(
            BN_ReLU_Conv(n2[1][0], kernel_size=1),
            BN_ReLU_Conv(n2[1][1], kernel_size=(7,1), padding=(3,0)),
            BN_ReLU_Conv(n2[1][2], kernel_size=(1,7), padding=(0,3)),
            BN_ReLU_Conv(n2[1][3], kernel_size=3)
        )
        
        # path3
        self.conv2 = BN_ReLU_Conv(n3, kernel_size=3, strides=2)
        self.maxp2 = gluon.nn.MaxPool2D(pool_size=2)
        
    def forward(self, X):
        p1 = self.b1(X)
        p1 = nd.concat(self.conv1(p1), self.maxp1(p1), dim=1)
        p2 = nd.concat(self.b2(p1), self.b3(p1), dim=1)
        p3 = nd.concat(self.conv2(p2), self.maxp2(p2))
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            
        return p3

In [5]:
# X = nd.random.normal(shape=(1, 3, 299, 299), ctx=ctx)
# stem = Stem([32, 32, 64, 96], [[64, 96], [64, 64, 64, 96]], 192, debug=True)
# y = stem(X)

In [6]:
class Inception_A(gluon.Block):
    def __init__(self, n1, n2, n3, n4, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = gluon.nn.AvgPool2D(pool_size=3, strides=1, padding=1)
        self.p1_2 = BN_ReLU_Conv(n1, kernel_size=1)
        self.p2_1 = BN_ReLU_Conv(n2, kernel_size=1)
        self.p3_1 = BN_ReLU_Conv(n3[0], kernel_size=1)
        self.p3_2 = BN_ReLU_Conv(n3[1], kernel_size=3, padding=1)
        self.p4_1 = BN_ReLU_Conv(n4[0], kernel_size=1)
        self.p4_2 = BN_ReLU_Conv(n4[1], kernel_size=3, padding=1)
        self.p4_3 = BN_ReLU_Conv(n4[2], kernel_size=3, padding=1)
        
    def forward(self, X):
        p1 = self.p1_2(self.p1_1(X))
        p2 = self.p2_1(X)
        p3 = self.p3_2(self.p3_1(X))
        p4 = self.p4_3(self.p4_2(self.p4_1(X)))
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            print("p_4 : ", p4.shape)
            
        out = nd.concat(p1, p2, p3, p4, dim=1)
        return out   

In [7]:
# inception_a = Inception_A(96, 96, [64, 96], [64, 96, 96], debug=True)
# X = nd.random.normal(shape=(1, 384, 35, 35), ctx=ctx)
# y = inception_a(X)
# y.shape

In [8]:
class Reduction_A(gluon.Block):
    def __init__(self, n2, n3, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = gluon.nn.MaxPool2D(pool_size=2)
        self.p2_1 = BN_ReLU_Conv(n2, kernel_size=3, strides=2)
        self.p3_1 = BN_ReLU_Conv(n3[0], kernel_size=1)
        self.p3_2 = BN_ReLU_Conv(n3[1], kernel_size=3, padding=1)
        self.p3_3 = BN_ReLU_Conv(n3[2], kernel_size=3, strides=2)
        
    def forward(self, X):
        p1 = self.p1_1(X)
        p2 = self.p2_1(X)
        p3 = self.p3_3(self.p3_2(self.p3_1(X)))
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            
        out = nd.concat(p1, p2, p3, dim=1)
        return out   

In [9]:
# reduction_a = Reduction_A(384, [192, 224, 256], debug=True) 
# X = nd.random.normal(shape=(1, 384, 35, 35), ctx=ctx)
# y = reduction_a(X)
# y.shape

In [10]:
class Inception_B(gluon.Block):
    def __init__(self, n1, n2, n3, n4, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = gluon.nn.AvgPool2D(pool_size=3, strides=1, padding=1)
        self.p1_2 = BN_ReLU_Conv(n1, kernel_size=1)
        self.p2_1 = BN_ReLU_Conv(n2, kernel_size=1)
        self.p3_1 = BN_ReLU_Conv(n3[0], kernel_size=1)
        self.p3_2 = BN_ReLU_Conv(n3[1], kernel_size=(1,7), padding=(0,3))
        self.p3_3 = BN_ReLU_Conv(n3[2], kernel_size=(7,1), padding=(3,0))
        self.p4_1 = BN_ReLU_Conv(n4[0], kernel_size=1)
        self.p4_2 = BN_ReLU_Conv(n4[1], kernel_size=(1,7), padding=(0,3))
        self.p4_3 = BN_ReLU_Conv(n4[2], kernel_size=(7,1), padding=(3,0))
        self.p4_4 = BN_ReLU_Conv(n4[3], kernel_size=(1,7), padding=(0,3))
        self.p4_5 = BN_ReLU_Conv(n4[4], kernel_size=(7,1), padding=(3,0))
        
    def forward(self, X):
        p1 = self.p1_2(self.p1_1(X))
        p2 = self.p2_1(X)
        p3 = self.p3_3(self.p3_2(self.p3_1(X)))
        p4 = self.p4_5(self.p4_4(self.p4_3(self.p4_2(self.p4_1(X)))))
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            print("p_4 : ", p4.shape)
            
        out = nd.concat(p1, p2, p3, p4, dim=1)
        return out   

In [11]:
# inception_b = Inception_B(128, 384, [192, 224, 256], [192, 192, 224, 224, 256], debug=True)
# X = nd.random.normal(shape=(1, 1024, 17, 17), ctx=ctx)
# y = inception_b(X)
# y.shape

In [12]:
class Reduction_B(gluon.Block):
    def __init__(self, n2, n3, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = gluon.nn.MaxPool2D(pool_size=2)
        self.p2_1 = BN_ReLU_Conv(n2[0], kernel_size=1)
        self.p2_2 = BN_ReLU_Conv(n2[1], kernel_size=3, strides=2)
        self.p3_1 = BN_ReLU_Conv(n3[0], kernel_size=1)
        self.p3_2 = BN_ReLU_Conv(n3[1], kernel_size=(1,7), padding=(0,3))
        self.p3_3 = BN_ReLU_Conv(n3[2], kernel_size=(7,1), padding=(3,0))
        self.p3_4 = BN_ReLU_Conv(n3[3], kernel_size=3, strides=2)
        
    def forward(self, X):
        p1 = self.p1_1(X)
        p2 = self.p2_2(self.p2_1(X))
        p3 = self.p3_4(self.p3_3(self.p3_2(self.p3_1(X))))
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            
        out = nd.concat(p1, p2, p3, dim=1)
        return out   

In [13]:
# reduction_b = Reduction_B([192, 192], [256, 256, 320, 320], debug=True) 
# X = nd.random.normal(shape=(1, 1024, 17, 17), ctx=ctx)
# y = reduction_b(X)
# y.shape

In [14]:
class Inception_C(gluon.Block):
    def __init__(self, n1, n2, n3, n4, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = gluon.nn.AvgPool2D(pool_size=3, strides=1, padding=1)
        self.p1_2 = BN_ReLU_Conv(n1, kernel_size=1)
        self.p2_1 = BN_ReLU_Conv(n2, kernel_size=1)
        self.p3_1 = BN_ReLU_Conv(n3[0], kernel_size=1)
        self.p3_2 = BN_ReLU_Conv(n3[1], kernel_size=(1,3), padding=(0,1))
        self.p3_3 = BN_ReLU_Conv(n3[2], kernel_size=(3,1), padding=(1,0))
        self.p4_1 = BN_ReLU_Conv(n4[0], kernel_size=1)
        self.p4_2 = BN_ReLU_Conv(n4[1], kernel_size=(1,3), padding=(0,1))
        self.p4_3 = BN_ReLU_Conv(n4[2], kernel_size=(3,1), padding=(1,0))
        self.p4_4 = BN_ReLU_Conv(n4[3], kernel_size=(1,3), padding=(0,1))
        self.p4_5 = BN_ReLU_Conv(n4[4], kernel_size=(3,1), padding=(1,0))
        
    def forward(self, X):
        p1 = self.p1_2(self.p1_1(X))
        p2 = self.p2_1(X)
        p3 = self.p3_1(X)
        p3 = nd.concat(self.p3_2(p3), self.p3_3(p3), dim=1)
        p4 = self.p4_3(self.p4_2(self.p4_1(X)))
        p4 = nd.concat(self.p4_4(p4), self.p4_5(p4), dim=1)
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            print("p_4 : ", p4.shape)
            
        out = nd.concat(p1, p2, p3, p4, dim=1)
        return out   

In [15]:
# inception_c = Inception_C(256, 256, [384, 256, 256], [384, 448, 512, 256, 256], debug=True)
# X = nd.random.normal(shape=(1, 1536, 8, 8), ctx=ctx)
# y = inception_c(X)
# y.shape

In [16]:
class Inception_V4(gluon.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super().__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            b1 = gluon.nn.Sequential()
            b1.add(Stem([32, 32, 64, 96], [[64, 96], [64, 64, 64, 96]], 192))
            
            b2 = gluon.nn.Sequential()
            for i in range(4):
                b2.add(Inception_A(96, 96, [64, 96], [64, 96, 96]))
            b2.add(Reduction_A(384, [192, 224, 256]))   
                
            b3 = gluon.nn.Sequential()
            for i in range(7):
                b3.add(Inception_B(128, 384, [192, 224, 256], [192, 192, 224, 224, 256]))
            b3.add(Reduction_B([192, 192], [256, 256, 320, 320]))
            
            b4 = gluon.nn.Sequential()
            for i in range(3):
                b4.add(Inception_C(256, 256, [384, 256, 256], [384, 448, 512, 256, 256]))
                
            b5 = gluon.nn.Sequential()
            b5.add(
                gluon.nn.AvgPool2D(pool_size=8),
                gluon.nn.Dropout(0.2),
            )
            
            b6 = gluon.nn.Sequential()
            b6.add(
                gluon.nn.Flatten(),
                gluon.nn.Dense(num_classes)
            )
        self.net = gluon.nn.Sequential()
        self.net.add(b1, b2, b3, b4, b5, b6)
            
    def forward(self, X):
        out = X
        for i, blk in enumerate(self.net):
            out = blk(out)
            if self.verbose:
                print("blk %d : %s."% (i+1, out.shape))
        return out

In [17]:
X = nd.random.normal(shape=(1, 3, 299, 299), ctx=ctx)
inception_v4 = Inception_V4(10, verbose=True)
inception_v4.initialize(ctx=ctx)
y = inception_v4(X)

blk 1 : (1, 384, 35, 35).
blk 2 : (1, 1024, 17, 17).
blk 3 : (1, 1536, 8, 8).
blk 4 : (1, 1536, 8, 8).
blk 5 : (1, 1536, 1, 1).
blk 6 : (1, 10).


## Inception-ResNet-V1


#### Inception-ResNet-V1以及Inception-ResNet-V2网络的架构：

两个网络的区别是每个模块的架构不相同，以及输出的channels不同，但网络的整体结构是相同的,我们这里只实现V1版本

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-19.png" width="320">

**Inception-ResNet定义的Stem模块：**

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-26.png" width="300">

**Inception-ResNet-A模块：**

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-27.png" width="400">

**输出尺寸减半模块(Reduction-A):**

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-22.png" width="400">

**Inception-ResNet-B模块：**

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-28.png" width="300">

**输出尺寸减半模块(Reduction-B):**

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-30.png" width="400">

**Inception-ResNet-C模块：**

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-29.png" width="350">

In [18]:
def CONV_BN_ReLU(channels, kernel_size, strides=1, padding=0):
    net = gluon.nn.Sequential()
    with net.name_scope():
        net.add(
            gluon.nn.Conv2D(channels, kernel_size=kernel_size, strides=strides, padding=padding),
            gluon.nn.BatchNorm(axis=1),
            gluon.nn.Activation('relu')
        ) 
    return net

**图14的Inception_ResNet_Stem**

In [19]:
def Inception_ResNet_Stem(n1, n2, n3, n4, n5, n6):
    stem = gluon.nn.Sequential()
    with stem.name_scope():
        stem.add(CONV_BN_ReLU(n1, kernel_size=3, strides=2))
        stem.add(CONV_BN_ReLU(n2, kernel_size=3))
        stem.add(CONV_BN_ReLU(n3, kernel_size=3, padding=1))
        stem.add(gluon.nn.MaxPool2D(pool_size=2))
        stem.add(CONV_BN_ReLU(n4, kernel_size=1))
        stem.add(CONV_BN_ReLU(n5, kernel_size=3))
        stem.add(CONV_BN_ReLU(n6, kernel_size=3, strides=2))
    return stem

In [20]:
X = nd.random.normal(shape=(1, 3, 299, 299), ctx=ctx)
stem = Inception_ResNet_Stem(32, 32, 64, 80, 192, 256)
stem.initialize(ctx=ctx)
y = stem(X)
y.shape

(1, 256, 35, 35)

**图10的Inception-ResNet-A**

In [21]:
class Inception_ResNet_A(gluon.Block):
    def __init__(self, n1, n2, n3, n_concat, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.conv1 = CONV_BN_ReLU(n1, kernel_size=1)
        
        self.b1 = gluon.nn.Sequential()
        self.b1.add(
            CONV_BN_ReLU(n2[0], kernel_size=1),
            CONV_BN_ReLU(n2[1], kernel_size=3, padding=1)
        )
        
        self.b2 = gluon.nn.Sequential()
        self.b2.add(
            CONV_BN_ReLU(n3[0], kernel_size=1),
            CONV_BN_ReLU(n3[1], kernel_size=3, padding=1),
            CONV_BN_ReLU(n3[2], kernel_size=3, padding=1)
        )
        
        self.conv2 = CONV_BN_ReLU(n_concat, kernel_size=1)
        
    def forward(self, X):
        p1 = self.conv1(X)
        p2 = self.b1(X)
        p3 = self.b2(X)
        residual = self.conv2(nd.concat(p1, p2, p3, dim=1))
        
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("p3 : ", p3.shape)
            print("residul : ", residual.shape)
        
        return nd.relu(residual + X)   

**测试**

In [22]:
ira = Inception_ResNet_A(32, [32, 32], [32, 48, 64], n_concat=256, debug=True)
ira.initialize(ctx=ctx)
X = nd.random.normal(shape=(1, 256, 35, 35), ctx=ctx)
y = ira(X)
y.shape

p1 :  (1, 32, 35, 35)
p2 :  (1, 32, 35, 35)
p3 :  (1, 64, 35, 35)
residul :  (1, 256, 35, 35)


(1, 256, 35, 35)

**图11的Inception-ResNet-B**

In [23]:
class Inception_ResNet_B(gluon.Block):
    def __init__(self, n1, n2, n_concat, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.conv1 = CONV_BN_ReLU(n1, kernel_size=1)
        
        self.b1 = gluon.nn.Sequential()
        self.b1.add(
            CONV_BN_ReLU(n2[0], kernel_size=1),
            CONV_BN_ReLU(n2[1], kernel_size=(1,7), padding=(0,3)),
            CONV_BN_ReLU(n2[2], kernel_size=(7,1), padding=(3,0))
        )
        
        self.conv2 = CONV_BN_ReLU(n_concat, kernel_size=1)
        
    def forward(self, X):
        p1 = self.conv1(X)
        p2 = self.b1(X)
        residual = self.conv2(nd.concat(p1, p2, dim=1))
        
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("residul : ", residual.shape)
        
        return nd.relu(residual + X)   

**测试**

In [24]:
irb = Inception_ResNet_B(128, [128, 128, 128], n_concat=896, debug=True)
irb.initialize(ctx=ctx)
X = nd.random.normal(shape=(1, 896, 17, 17), ctx=ctx)
y = irb(X)
y.shape

p1 :  (1, 128, 17, 17)
p2 :  (1, 128, 17, 17)
residul :  (1, 896, 17, 17)


(1, 896, 17, 17)

**图12的Reduction-B**

** Reduction-A的结构与上面是一样的，但是Reduction-B的结构是不一样的，所以重新写Reduction-B**

In [25]:
class IR_Reduction_B(gluon.Block):
    def __init__(self, n2, n3, n4, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug 
        self.p1_1 = gluon.nn.MaxPool2D(pool_size=2)
        self.p2_1 = CONV_BN_ReLU(n2[0], kernel_size=1)
        self.p2_2 = CONV_BN_ReLU(n2[1], kernel_size=3, strides=2)
        self.p3_1 = CONV_BN_ReLU(n3[0], kernel_size=1)
        self.p3_2 = CONV_BN_ReLU(n3[1], kernel_size=3, strides=2)
        self.p4_1 = CONV_BN_ReLU(n4[0], kernel_size=1)
        self.p4_2 = CONV_BN_ReLU(n4[1], kernel_size=3, padding=1)
        self.p4_3 = CONV_BN_ReLU(n4[2], kernel_size=3, strides=2)
        
    def forward(self, X):
        p1 = self.p1_1(X)
        p2 = self.p2_2(self.p2_1(X))
        p3 = self.p3_2(self.p3_1(X))
        p4 = self.p4_3(self.p4_2(self.p4_1(X)))
        
        if self.debug:
            print("p_1 : ", p1.shape)
            print("p_2 : ", p2.shape)
            print("p_3 : ", p3.shape)
            print("p_4 : ", p4.shape)
            
        out = nd.concat(p1, p2, p3, p4, dim=1)
        return out   

**测试**

In [26]:
ir_reduction_b = IR_Reduction_B([256, 384], [256, 256], [256, 256, 256], debug=True) 
ir_reduction_b.initialize(ctx=ctx)
X = nd.random.normal(shape=(1, 896, 17, 17), ctx=ctx)
y = ir_reduction_b(X)
y.shape

p_1 :  (1, 896, 8, 8)
p_2 :  (1, 384, 8, 8)
p_3 :  (1, 256, 8, 8)
p_4 :  (1, 256, 8, 8)


(1, 1792, 8, 8)

** 图13的Inception_ResNet_C**

In [27]:
class Inception_ResNet_C(gluon.Block):
    def __init__(self, n1, n2, n_concat, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.conv1 = CONV_BN_ReLU(n1, kernel_size=1)
        
        self.b1 = gluon.nn.Sequential()
        self.b1.add(
            CONV_BN_ReLU(n2[0], kernel_size=1),
            CONV_BN_ReLU(n2[1], kernel_size=(1,3), padding=(0,1)),
            CONV_BN_ReLU(n2[2], kernel_size=(3,1), padding=(1,0))
        )
        
        self.conv2 = CONV_BN_ReLU(n_concat, kernel_size=1)
        
    def forward(self, X):
        p1 = self.conv1(X)
        p2 = self.b1(X)
        residual = self.conv2(nd.concat(p1, p2, dim=1))
        
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("residul : ", residual.shape)
        
        return nd.relu(residual + X)  

**测试**

In [28]:
irc = Inception_ResNet_C(192, [192, 192, 192], n_concat=1792, debug=True)
irc.initialize(ctx=ctx)
X = nd.random.normal(shape=(1, 1792, 17, 17), ctx=ctx)
y = irc(X)
y.shape

p1 :  (1, 192, 17, 17)
p2 :  (1, 192, 17, 17)
residul :  (1, 1792, 17, 17)


(1, 1792, 17, 17)

In [29]:
class Inception_ResNet_V1(gluon.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super().__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            b1 = gluon.nn.Sequential()
            b1.add(Inception_ResNet_Stem(32, 32, 64, 80, 192, 256))
            
            b2 = gluon.nn.Sequential()
            for i in range(5):
                b2.add(Inception_ResNet_A(32, [32, 32], [32, 48, 64], n_concat=256))
            b2.add(Reduction_A(384, [192, 192, 256]))   
                
            b3 = gluon.nn.Sequential()
            for i in range(10):
                b3.add(Inception_ResNet_B(128, [128, 128, 128], n_concat=896))
            b3.add(IR_Reduction_B([256, 384], [256, 256], [256, 256, 256]))
            
            b4 = gluon.nn.Sequential()
            for i in range(5):
                b4.add(Inception_ResNet_C(192, [192, 192, 192], n_concat=1792))
                
            b5 = gluon.nn.Sequential()
            b5.add(
                gluon.nn.AvgPool2D(pool_size=8),
                gluon.nn.Dropout(0.2),
            )
            
            b6 = gluon.nn.Sequential()
            b6.add(
                gluon.nn.Flatten(),
                gluon.nn.Dense(num_classes)
            )
        self.net = gluon.nn.Sequential()
        self.net.add(b1, b2, b3, b4, b5, b6)
            
    def forward(self, X):
        out = X
        for i, blk in enumerate(self.net):
            out = blk(out)
            if self.verbose:
                print("blk %d : %s."% (i+1, out.shape))
        return out

In [30]:
X = nd.random.normal(shape=(1, 3, 299, 299), ctx=ctx)
inception_resnet_v1 = Inception_ResNet_V1(10, verbose=True)
inception_resnet_v1.initialize(ctx=ctx)
y = inception_resnet_v1(X)

blk 1 : (1, 256, 35, 35).
blk 2 : (1, 896, 17, 17).
blk 3 : (1, 1792, 8, 8).
blk 4 : (1, 1792, 8, 8).
blk 5 : (1, 1792, 1, 1).
blk 6 : (1, 10).


In [32]:
import utils
from time import time

inception_resnet_v1 = inception_resnet_v1(10, verbose=False)
inception_resnet_v1.initialize(ctx=ctx)

epochs = 10
learning_rate = .1
trainer = gluon.Trainer(inception_resnet_v1.collect_params(), 'Adam', {'learning_rate': learning_rate})
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

batch_size = 64
train_data, test_data = utils.load_dataset(batch_size, resize=299, data_type='cifar10')

niter = 0
moving_loss = .0
smoothing_constant = .9

for epoch in range(epochs):
    for data, label in train_data:
        with autograd.record():
            output = inception_resnet_v1(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()